1. What do you think we should do with these null values in the `fresh_segments.interest_metrics`

In [9]:
UPDATE fresh_segments.interest_metrics
SET
    month_year = COALESCE(month_year,'NO RECORD'),
    interest_id = COALESCE(interest_id,'NO RECORD'),
    _month = COALESCE(_month,0),
    _year = COALESCE(_year,0)


(13940 rows affected)

Total execution time: 00:00:00.396

2. What is count of records in the `fresh_segments.interest_metrics` for each `month_year` value sorted in chronological order (earliest to latest) with the null values appearing first?

In [14]:
SELECT 
    CAST(CAST(month_year + '-01' AS nvarchar(10)) AS date) AS ConvertedDate , 
    COUNT(*) AS number_of_records 
FROM fresh_segments.interest_metrics
WHERE month_year != 'NO RECORD'
GROUP BY month_year
ORDER BY month_year


(14 rows affected)

Total execution time: 00:00:00.063

ConvertedDate,number_of_records
2018-07-01,729
2018-08-01,767
2018-09-01,780
2018-10-01,857
2018-11-01,928
2018-12-01,995
2019-01-01,973
2019-02-01,1121
2019-03-01,1136
2019-04-01,1099


3. How many `interest_id` values exist in the `fresh_segments.interest_metrics` table but not in the `fresh_segments.interest_map` table? What about the other way around?

In [10]:
SELECT COUNT(*) as [interest_id values exist]
FROM
    (SELECT 
        DISTINCT(interest_id)
    FROM fresh_segments.interest_metrics
    WHERE interest_id != 'NO RECORD'
    EXCEPT
    SELECT
        DISTINCT(id)
    FROM fresh_segments.interest_map) AS RESULT

---> tìm duy nhất được 1 interest_id

-- what about the other way around?
SELECT COUNT(*) as [interest_id values exist in contrast]
FROM 
    (SELECT
        DISTINCT(id)
    FROM fresh_segments.interest_map
    EXCEPT
    SELECT 
        DISTINCT(interest_id)
    FROM fresh_segments.interest_metrics
    WHERE interest_id != 'NO RECORD') AS RESULT

---> tìm được 7 ids ngược lại

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.143

interest_id values exist
1


interest_id values exist in contrast
7


4. Summarise the `id` values in the `fresh_segments.interest_map` by its total record count in this table

In [11]:
SELECT COUNT(ID) as TOTAL_RECORD
FROM fresh_segments.interest_map;

(1 row affected)

Total execution time: 00:00:00.011

TOTAL_RECORD
1208


5. What sort of table join should we perform for our analysis and why? Check your logic by checking the rows where `interest_id = 21246` in your joined output and include all columns from `fresh_segments.interest_metrics` and all columns from `fresh_segments.interest_map` except from the `id` column.

In [9]:

SELECT TOP 10
     CAST(CAST(A.month_year + '-01' AS nvarchar(10)) AS date) AS ConvertedDate , A.interest_id , B.interest_name , A.composition , A.index_value , A.ranking , A.percentile_ranking,
        B.interest_summary , B.created_at , B.last_modified
FROM fresh_segments.interest_metrics A JOIN fresh_segments.interest_map B
    ON A.interest_id = B.id
WHERE A.month_year != 'NO RECORD' OR A._month != 0 OR A._year != 0



--- nên sort thứ tự theo 2 trường interest_name (hoặc interest_id) và month_year (cũ nhất - mới nhất).
---- Theo interest_name (hoặc interest_id) chúng ta cần nhóm interest_name lại với nhau để dễ dàng quan sát những kết quả về metric của online ad mà khách hàng đạt được
--- (về thời gian tương tác, khoảng thời gian từ lúc xây dựng 1 online ad cho đến khi đạt được những kết quả metric đầu tiên,... ).
---- Kế tiếp là sort theo month_year là vì chúng ta có thể quan sát trung bình 1 online ad sẽ nhận được sự tương tác của khách hàng trong bao lâu, theo thứ tự earliest - latest
--- là vì chúng ta sẽ quan sát được xu hướng tăng trưởng hay giảm sút trong kết quả các chỉ số

(10 rows affected)

Total execution time: 00:00:00.059

ConvertedDate,interest_id,interest_name,composition,index_value,ranking,percentile_ranking,interest_summary,created_at,last_modified
2018-07-01,32486,Vacation Rental Accommodation Researchers,11.89,6.19,1,99.86,People researching and booking rentals accommodations for vacations.,2018-06-29 12:55:03.000,2018-06-29 12:55:03.000
2018-07-01,6106,Luxury Second Home Owners,9.93,5.31,2,99.73,High income individuals with more than one home.,2017-03-27 16:59:29.000,2018-05-23 11:30:12.000
2018-07-01,18923,Online Home Decor Shoppers,10.85,5.29,3,99.59,Consumers shopping online for home decor available for delivery.,2018-04-19 18:25:02.000,2018-04-19 18:25:02.000
2018-07-01,6344,Hair Care Shoppers,10.32,5.1,4,99.45,Consumers researching trends and purchasing hair and beauty products.,2017-05-15 13:04:55.000,2018-05-31 22:11:37.000
2018-07-01,100,Nutrition Conscious Eaters,10.77,5.04,5,99.31,Consumer reading about healthy eating options.,2016-05-26 14:57:59.000,2018-05-23 11:30:13.000
2018-07-01,69,Healthy Eaters,10.82,5.03,6,99.18,People researching healthy eating options.,2016-05-26 14:57:59.000,2018-05-23 11:30:12.000
2018-07-01,79,Luxury Travel Researchers,11.21,4.97,7,99.04,Consumers reading online reviews of luxury travel options.,2016-05-26 14:57:59.000,2018-05-23 11:30:12.000
2018-07-01,6111,Wine Lovers,10.71,4.83,8,98.9,Consumers researching wine and purchasing alcohol online. These consumers are more likely to purchase wine and visit vineyards.,2017-03-27 16:59:29.000,2017-12-07 12:35:47.000
2018-07-01,6214,Home Remodelers,9.71,4.83,8,98.9,People researching techniques and resources for home remodels.,2017-03-27 16:59:29.000,2018-05-23 11:30:12.000
2018-07-01,19422,Home Design and Living Publication Readers,10.11,4.81,10,98.63,People reading publications focused on design and living at home.,2018-05-08 11:55:03.000,2018-05-08 11:55:03.000


Interest Analysis

1. Which interests have been present in all `month_year` dates in our dataset?

In [8]:
SELECT TOP 10
    interest_id,
    COUNT(month_year) AS TOTAL_MONTH
FROM fresh_segments.interest_metrics
GROUP BY interest_id
HAVING COUNT(month_year) = (
    SELECT COUNT(DISTINCT(month_year))
    FROM fresh_segments.interest_metrics
    WHERE month_year != 'no record'
)
-- TOTAL_MONTH: khoảng thời gian dài nhất được ghi nhận mà 1 online ad nhận được sự tương tác từ khách hàng

(10 rows affected)

Total execution time: 00:00:00.106

interest_id,TOTAL_MONTH
100,14
10008,14
101,14
102,14
10249,14
10250,14
10284,14
10326,14
10351,14
107,14


2. Using this same `total_months` measure - calculate the cumulative percentage of all records starting at 14 months - which `total_months` value passes the 90% cumulative percentage value?

In [7]:
DECLARE @total FLOAT
SELECT 
     @total = SUM(TOTAL_MONTH) 
FROM (
    SELECT 
        interest_id,
        COUNT(month_year) AS TOTAL_MONTH
    FROM fresh_segments.interest_metrics
    WHERE month_year != 'no record'
    GROUP BY interest_id
) AS result;

WITH CTE AS 
    (SELECT 
        interest_id,
        TOTAL_MONTH,
        -- Tính tổng tích lũy (Cumulative Sum) từ dòng hiện tại đến tất cả các dòng trước đó
        SUM(TOTAL_MONTH) OVER (ORDER BY TOTAL_MONTH DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumulative_sum
    FROM (
        SELECT 
            interest_id,
            COUNT(month_year) AS TOTAL_MONTH
        FROM fresh_segments.interest_metrics
        WHERE month_year != 'no record'
        GROUP BY interest_id
    ) AS RESULT2)
, CTE1 AS
	(SELECT 
		interest_id,
		TOTAL_MONTH,
		cumulative_sum,
		ROUND(cast( (cumulative_sum / @total) as float)*100,2) AS cumulative_percentage
	FROM CTE) 

SELECT TOP 10
	interest_id,
	TOTAL_MONTH,
	cumulative_percentage
FROM CTE1
WHERE cumulative_percentage > 90

-- Trong tập dữ liệu, thời dài nhất mà một online ad nhận được sự tương tác từ khách hàng là 14 tháng.
-- Trong đó, chiếm 10% là các online ad có thời gian tương tác với khách hàng ít hơn 8 tháng

(10 rows affected)

Total execution time: 00:00:00.257

interest_id,TOTAL_MONTH,cumulative_percentage
57,8,90.06
6363,8,90.12
5900,7,90.18
4847,7,90.23
45725,7,90.29
45673,7,90.34
45674,7,90.4
45675,7,90.45
45676,7,90.51
45677,7,90.56


3. If we were to remove all `interest_id` values which are lower than the `total_months` value we found in the previous question - how many total data points would we be removing?

In [20]:
SELECT 
    COUNT(T1.interest_id) AS [NUMBER OF RECORDS REMOVED]
FROM fresh_segments.interest_metrics T1 JOIN (
    SELECT 
        interest_id,
        COUNT(month_year) AS 'TOTAL MONTH'
    FROM fresh_segments.interest_metrics
    GROUP BY INTEREST_ID
    HAVING COUNT(month_year) <= 8    
) T2
    ON T1.interest_id = T2.interest_id


(1 row affected)

Total execution time: 00:00:00.049

NUMBER OF RECORDS REMOVED
1827


4. Does this decision make sense to remove these data points from a business perspective? Use an example where there are all 14 months present to a removed `interest` example for your arguments - think about what it means to have less months present from a segment perspective.

Từ khía cạnh doanh nghiệp:

- 1 strategic online ad có thể nằm trong 1 chiến lược hoạt động của business customer hoặc cùng 1 segment marketing online, giả sử một sản phẩm mới được tung ra và sản phẩm online ad được dùng để quan sát cảm nhận người tiêu dùng đối với sản phẩm trong thời gian ngắn, việc remove những records này đồng nghĩa với việc Fresh_Segments đưa ra nhìn nhận chủ quan khi cho rằng những online ad này không hiệu quả vì thời gian tương tác với khách hàng quá ít hoặc số lượng online ad nằm trong 1 phân khúc cụ thể không quá nhiều.

Từ khía cạnh người phân tích dữ liệu:

- Cần tìm hiểu xem những ad online này có cùng thuộc 1 business customer, chiến lược phía sau cũng như mục đích của business customer khi launch những strategic online ad này là gì,..

- Những metrics của các online ad này có phải outliers value trong dataset hay không? (Bởi vì outliers có thể ảnh hưởng cả tích cực lẫn tiêu cực đến quá trình phân tích)

--> Việc đưa ra quyết định loại bỏ ngay những records có thời gian tương tác với khách hàng ít hơn 8 tháng là không hợp lý.

5. After removing these interests - how many unique interests are there for each month?

In [18]:
WITH CTE1 AS 
(    SELECT 
        interest_id,
        COUNT(month_year) AS TOTAL_MONTH
    FROM fresh_segments.interest_metrics
    WHERE month_year != 'NO RECORD'
    GROUP BY interest_id
    HAVING COUNT(month_year) <= 8)
, CTE2 AS 
(    SELECT 
        CAST(CAST(month_year + '-01' AS nvarchar(10)) AS date) AS ConvertedDate,
        interest_id
    FROM fresh_segments.interest_metrics
    WHERE interest_id NOT IN (
        SELECT 
            interest_id
        FROM CTE1
    ) AND month_year != 'NO RECORD')

SELECT 
    ConvertedDate,
    COUNT(DISTINCT(interest_id)) AS [NUMBER OF UNIQUE INTEREST]
FROM CTE2
GROUP BY ConvertedDate
ORDER BY ConvertedDate;

(14 rows affected)

Total execution time: 00:00:00.349

ConvertedDate,NUMBER OF UNIQUE INTEREST
2018-07-01,681
2018-08-01,720
2018-09-01,744
2018-10-01,792
2018-11-01,848
2018-12-01,884
2019-01-01,861
2019-02-01,885
2019-03-01,885
2019-04-01,861


Segment Analysis

1. Using our filtered dataset by removing the interests with less than 6 months worth of data, which are the top 10 and bottom 10 interests which have the largest composition values in any `month_year`? Only use the maximum composition value for each interest but you must keep the corresponding `month_year`

In [11]:
CREATE VIEW filtered_data as
    WITH CTE1 AS 
    (    SELECT
            interest_id,
            COUNT(month_year) AS TOTAL_MONTH
        FROM fresh_segments.interest_metrics
        WHERE month_year != 'NO RECORD'
        GROUP BY interest_id
        HAVING COUNT(month_year) > 6)
    , CTE2 AS (
        SELECT 
            CAST(CAST(month_year + '-01' AS nvarchar(10)) AS date) AS ConvertedDate,
            interest_id,
            composition,
            index_value,
            ranking,
            percentile_ranking
        FROM fresh_segments.interest_metrics
        WHERE interest_id IN 
            (
                SELECT interest_id
                FROM CTE1
            ) AND month_year != 'NO RECORD'
    )
    SELECT
        ConvertedDate,
        interest_id,
        composition,
        index_value,
        ranking,
        percentile_ranking  
    FROM CTE2

Commands completed successfully.

Total execution time: 00:00:00.042

In [13]:
;WITH CTE0 AS
    (SELECT 
        ConvertedDate,
        interest_id,
        composition,
        DENSE_RANK() OVER (PARTITION BY interest_id ORDER BY composition DESC) AS [COMPOSITION RANK TEMP]
    FROM filtered_data),
CTE1 AS (
    SELECT
        ConvertedDate,
        interest_id,
        composition
    FROM CTE0 
    WHERE [COMPOSITION RANK TEMP] = 1 --- Lấy những composition cao nhất của mỗi interest_id
), --- Lấy top 10
CTE2 AS (
    SELECT TOP 10
        ConvertedDate,
        interest_id,
        composition,
        DENSE_RANK() OVER (ORDER BY composition DESC) AS [COMPOSITION RANK]
    FROM CTE1
), --- lấy bottom 10
CTE3 AS (
    SELECT
        ConvertedDate,
        interest_id,
        composition,
        DENSE_RANK() OVER (ORDER BY composition DESC) AS [COMPOSITION RANK]
    FROM CTE1
    ORDER BY composition DESC
    OFFSET ((SELECT COUNT(interest_id) FROM CTE1) - 10) ROWS
    FETCH NEXT 10 ROWS ONLY
)
--- nối top 10 với bottom 10
SELECT
    ConvertedDate,
    interest_id,
    [COMPOSITION RANK]
FROM CTE2
UNION
SELECT
    ConvertedDate,
    interest_id,
    [COMPOSITION RANK]
FROM CTE3
ORDER BY [COMPOSITION RANK]

(20 rows affected)

Total execution time: 00:00:00.440

ConvertedDate,interest_id,COMPOSITION RANK
2018-12-01,21057,1
2018-07-01,6284,2
2018-07-01,39,3
2018-07-01,77,4
2018-10-01,12133,5
2018-12-01,5969,6
2018-07-01,171,7
2018-07-01,4898,8
2018-07-01,6286,9
2018-07-01,4,10


2. Which 5 interests had the lowest average `ranking` value?

In [12]:
WITH CTE0 AS 
    (SELECT 
        interest_id,
        AVG(ranking) AS AVERAGE_VALUE
    FROM filtered_data
    GROUP BY interest_id , ranking)

SELECT TOP 5
    interest_id,
    AVERAGE_VALUE AS LOWEST_AVG_RANKING,
    DENSE_RANK() OVER (ORDER BY AVERAGE_VALUE ASC) AS AVERAGE_RANKING_VALUE
FROM CTE0 

(5 rows affected)

Total execution time: 00:00:07.795

interest_id,LOWEST_AVG_RANKING,AVERAGE_RANKING_VALUE
6285,1,1
32486,1,1
6218,1,1
41548,1,1
42203,2,2


3. Which 5 interests had the largest standard deviation in their `percentile_ranking` value?

In [15]:
WITH CTE0 AS 
    (SELECT 
        interest_id,
        percentile_ranking,
        ROUND(
            AVG(percentile_ranking) OVER (PARTITION BY interest_id),2) AS AVG_BY_PERCENTILE_RANKING,
        ROUND(
            POWER(
                (percentile_ranking - ROUND(AVG(percentile_ranking) OVER (PARTITION BY interest_id),2)),2),2) AS SQUARE_OF_DIFF
    FROM FILTERED_DATA),
CTE1 AS 
    (SELECT
        interest_id,
        percentile_ranking,
        AVG_BY_PERCENTILE_RANKING,
        SQUARE_OF_DIFF,
        ROUND(
            SQRT(
                SUM(SQUARE_OF_DIFF) OVER (PARTITION BY interest_id) / 
                    COUNT(interest_id) OVER (PARTITION BY interest_id)),2) AS STANDARD_DEVIATION
    FROM CTE0),
CTE2 AS
    (
        SELECT
            DISTINCT(STANDARD_DEVIATION), 
            interest_id
        FROM CTE1
        ORDER BY STANDARD_DEVIATION DESC
        OFFSET 0 ROW
        FETCH NEXT 5 ROWS ONLY
    )

SELECT 
    interest_id,
    STANDARD_DEVIATION
FROM CTE2

(5 rows affected)

Total execution time: 00:00:00.250

interest_id,STANDARD_DEVIATION
38992,27.95
43546,26.01
6087,23.68
103,23.45
4855,23.14


4. For the 5 interests found in the previous question - what was minimum and maximum `percentile_ranking` values for each interest and its corresponding `year_month` value? Can you describe what is happening for these 5 interests?

In [15]:
WITH CTE0 AS 
    (SELECT 
        interest_id,
        percentile_ranking,
        ROUND(
            AVG(percentile_ranking) OVER (PARTITION BY interest_id),2) AS AVG_BY_PERCENTILE_RANKING,
        ROUND(
            POWER(
                (percentile_ranking - ROUND(AVG(percentile_ranking) OVER (PARTITION BY interest_id),2)),2),2) AS SQUARE_OF_DIFF
    FROM FILTERED_DATA),
CTE1 AS 
    (SELECT
        interest_id,
        percentile_ranking,
        AVG_BY_PERCENTILE_RANKING,
        SQUARE_OF_DIFF,
        ROUND(
            SQRT(
                SUM(SQUARE_OF_DIFF) OVER (PARTITION BY interest_id) / 
                    COUNT(interest_id) OVER (PARTITION BY interest_id)),2) AS STANDARD_DEVIATION
    FROM CTE0),
CTE2 AS
    (SELECT
        DISTINCT(STANDARD_DEVIATION),
        interest_id
    FROM CTE1
    ORDER BY STANDARD_DEVIATION DESC
    OFFSET 0 ROW
    FETCH NEXT 5 ROWS ONLY),
CTE3 AS (
    SELECT 
        DISTINCT(interest_id),
        MIN(percentile_ranking) OVER (PARTITION BY interest_id ) AS MIN_PERCENTILE_RANKING,
        MAX(percentile_ranking) OVER (PARTITION BY interest_id ) AS MAX_PERCENTILE_RANKING
    FROM filtered_data
    WHERE interest_id IN 
        (
            SELECT 
                interest_id
            FROM CTE2 
        )
),
CTE4 AS --- Lấy những month_year của interest_id percentile_rank thấp nhất
    (
        SELECT 
        T1.interest_id , T2.ConvertedDate AS MONTH_YEAR_MIN_PERCENTILE, T1.MIN_PERCENTILE_RANKING
    FROM CTE3 T1 JOIN filtered_data T2
        ON T1.interest_id = T2.interest_id AND T1.MIN_PERCENTILE_RANKING = T2.percentile_ranking
    ),
CTE5 AS --- Lấy những month_year của interest_id percentile_rank cao nhất
    (
        SELECT 
            T1.interest_id , T2.ConvertedDate AS MONTH_YEAR_MAX_PERCENTILE, T1.MAX_PERCENTILE_RANKING
        FROM CTE3 T1 JOIN filtered_data T2
            ON T1.interest_id = T2.interest_id AND T1.MAX_PERCENTILE_RANKING = T2.percentile_ranking 
    )

SELECT 
    U1.interest_id ,
    U2.MONTH_YEAR_MAX_PERCENTILE,
    U2.MAX_PERCENTILE_RANKING,
    U1.MONTH_YEAR_MIN_PERCENTILE,
    U1.MIN_PERCENTILE_RANKING,
    DATEDIFF(MONTH,U2.MONTH_YEAR_MAX_PERCENTILE,U1.MONTH_YEAR_MIN_PERCENTILE) AS [TIME_INTERVAL(MONTH)]
FROM CTE4 U1 JOIN CTE5 U2 
    ON U1.interest_id = U2.interest_id 

--- càng về sau, tỷ lệ về thứ hạng index_value của các interest này đều giảm mạnh. Trong vòng 1 năm, mức độ tương tác của khách hàng đối với online ad này giảm đáng kể.

(5 rows affected)

Total execution time: 00:01:04.419

interest_id,MONTH_YEAR_MAX_PERCENTILE,MAX_PERCENTILE_RANKING,MONTH_YEAR_MIN_PERCENTILE,MIN_PERCENTILE_RANKING,TIME_INTERVAL(MONTH)
103,2018-07-01,95.61,2019-07-01,18.75,12
4855,2018-07-01,88.2,2018-11-01,13.15,4
6087,2018-07-01,81.89,2019-02-01,0.71,7
38992,2018-11-01,82.44,2019-07-01,2.2,8
43546,2019-03-01,73.15,2019-06-01,5.7,3


5. How would you describe our customers in this segment based off their composition and ranking values? What sort of products or services should we show to these customers and what should we avoid?

In [4]:
--- average ranking như thế nào? 
--- Engagement time như thế nào?
--- Thời gian từ lúc launch online ad cho đến khi đạt được max Composition là bao lâu?
WITH CTE0 AS 
    (SELECT
        DISTINCT(interest_name) AS INTEREST_NAME,
        (MAX(T1.RANKING) OVER (PARTITION BY CAST(T2.interest_name AS VARCHAR(255))) - MIN(T1.ranking) OVER (PARTITION BY CAST(T2.interest_name AS VARCHAR(255)))) AS RANKING_RANGE
    FROM filtered_data T1 JOIN fresh_segments.interest_map T2 
        ON T1.interest_id = T2.id),
CTE1 AS
    (
    SELECT
        T2.interest_name,
        CAST(CAST(T1.month_year + '-01' AS nvarchar(10)) AS date) AS ConvertedDate
    FROM filtered_data T1 JOIN fresh_segments.interest_map T2
        ON T1.interest_id = T2.id
    ),
CTE2 AS 
    (
        SELECT
            DISTINCT(interest_name),
            DATEDIFF(MONTH,
                MIN(ConvertedDate) OVER (PARTITION BY interest_name ORDER BY ConvertedDate),
                    MAX(ConvertedDate) OVER (PARTITION BY interest_name ORDER BY ConvertedDate)) AS ENGAGEMENT_TIME
        FROM CTE1 
    ),
CTE4 AS 
    (    SELECT 
            DISTINCT(interest_name) AS INTEREST_NAME,
            T2.created_at,
            MAX(T1.composition) OVER (PARTITION BY CAST(T2.interest_name AS VARCHAR(255)) ORDER BY CAST(T2.interest_name AS VARCHAR(255))) MAX_COMPOSITION
        FROM filtered_data T1 JOIN fresh_segments.interest_map T2
            ON T1.interest_id = T2.id),
CTE5 AS
    (SELECT 
        D3.INTEREST_NAME,
        D3.created_at,
        D1._month,
        D1._year,
        D3.MAX_COMPOSITION
    FROM (filtered_data D1 JOIN fresh_segments.interest_map D2 
        ON D1.interest_id = D2.id) JOIN CTE4 D3 
            ON D2.interest_name = D3.INTEREST_NAME AND D3.MAX_COMPOSITION = D1.composition)

SELECT TOP 10
    P1.INTEREST_NAME,
    P3.RANKING_RANGE,
    P2.ENGAGEMENT_TIME,
    (CASE 
        WHEN P1._year = YEAR(P1.created_at) THEN P1._month - MONTH(P1.created_at)
            ELSE (12 - MONTH(P1.created_at)) + P1._month END) AS [HIGHEST COMPOSITION TIME(MONTH)]
FROM (CTE5 P1 JOIN CTE2 P2 
    ON P1.INTEREST_NAME = P2.INTEREST_NAME) JOIN CTE0 P3
        ON P1.INTEREST_NAME = P3.INTEREST_NAME
ORDER BY [HIGHEST COMPOSITION TIME(MONTH)] ASC ,
            ENGAGEMENT_TIME DESC ,
                RANKING_RANGE ASC

--- có thể miêu tả khách hàng theo 3 tiêu chí đó chính
-- RANKING RANGE: cho biết phạm vi của ranking của các online ad, nếu có giá trị lớn, các thứ hạng của online ad này phân tán rộng, có sự khác biệ1 nhiều giữa thứ hạng lớn nhất và nhỏ nhất & ngược lại
-- ENGAGEMENT TIME: thời gian tối đa online ad nhận được sự tương tác từ khách hàng
-- HIGHEST COMPOSITION TIME: khoảng thời gian mà online ad đạt được max Composition


--- Dựa trên các chỉ tiêu này, có thể sắp xếp những online ad có Highest Composition Time ngắn nhất, sau đó là đến những online ad có Engagement Time lâu nhất.

(10 rows affected)

Total execution time: 00:00:20.831

INTEREST_NAME,RANKING_RANGE,ENGAGEMENT_TIME,HIGHEST COMPOSITION TIME(MONTH)
Major Airline Customers,169,13,0
Romantics,515,13,0
Sun Protection Shoppers,76,12,0
Major Airline Customers,169,12,0
Jazz Festival Enthusiasts,210,12,0
Toronto Blue Jays Fans,311,12,0
Candy Shoppers,356,12,0
Pittsburgh Steelers Fans,467,12,0
Cookie Eaters,491,12,0
New England Patriots Fans,498,12,0


Index Analysis
1. What is the top 10 interests by the average composition for each month?

In [17]:
WITH CTE0 AS 
    (SELECT 
        ConvertedDate,
        interest_id,
        AVG(composition) OVER (PARTITION BY ConvertedDate , interest_id ORDER BY composition DESC) AS AVG_COMPOSITION
    FROM filtered_data
    ),
CTE1 AS
    (
    SELECT 
        ConvertedDate,
        interest_id,
        AVG_COMPOSITION,
        DENSE_RANK() OVER (PARTITION BY ConvertedDate ORDER BY AVG_COMPOSITION DESC) AS AVG_COMPOSITION_RANKING
    FROM CTE0
    )

SELECT TOP 10
    ConvertedDate,
    interest_id,
    AVG_COMPOSITION,
    AVG_COMPOSITION_RANKING
FROM CTE1
WHERE AVG_COMPOSITION_RANKING <= 10


(10 rows affected)

Total execution time: 00:00:00.414

ConvertedDate,interest_id,AVG_COMPOSITION,AVG_COMPOSITION_RANKING
2018-07-01,6284,18.82,1
2018-07-01,39,17.44,2
2018-07-01,77,17.19,3
2018-07-01,171,14.91,4
2018-07-01,4898,14.23,5
2018-07-01,6286,14.1,6
2018-07-01,4,13.97,7
2018-07-01,17786,13.67,8
2018-07-01,6184,13.35,9
2018-07-01,4897,12.93,10


2. For all of these top 10 interests - which interest appears the most often?

In [18]:
WITH CTE0 AS 
    (SELECT 
        ConvertedDate,
        interest_id,
        AVG(composition) OVER (PARTITION BY ConvertedDate , interest_id ORDER BY composition DESC) AS AVG_COMPOSITION
    FROM filtered_data
    ),
CTE1 AS
    (
    SELECT 
        ConvertedDate,
        interest_id,
        AVG_COMPOSITION,
        DENSE_RANK() OVER (PARTITION BY ConvertedDate ORDER BY AVG_COMPOSITION DESC) AS AVG_COMPOSITION_RANKING
    FROM CTE0
    ),
CTE2 AS
    (
    SELECT 
        ConvertedDate,
        interest_id,
        AVG_COMPOSITION,
        AVG_COMPOSITION_RANKING
    FROM CTE1
    WHERE AVG_COMPOSITION_RANKING <= 10
    ),
CTE3 AS
    (
    SELECT 
        interest_id,
        COUNT(ConvertedDate) AS COUNT 
    FROM CTE2
    GROUP BY interest_id
    ),
CTE4 AS 
    (
    SELECT 
        interest_id,
        COUNT,
        DENSE_RANK() OVER (ORDER BY COUNT DESC) AS RANK
    FROM CTE3
    )

SELECT 
    interest_id 
FROM CTE4
WHERE RANK = 1

(3 rows affected)

Total execution time: 00:00:00.471

interest_id
12133
5969
6284


3. What is the average of the average composition for the top 10 interests for each month?

In [24]:
WITH CTE AS 
(
    SELECT
        ConvertedDate,
        interest_id,
        AVG(composition) OVER (PARTITION BY ConvertedDate, interest_id ORDER BY ConvertedDate) AS AVG_COMPOSITION
    FROM filtered_data
),CTE1 AS
(
    SELECT
        ConvertedDate,
        interest_id,
        AVG_COMPOSITION,
        DENSE_RANK() OVER (PARTITION BY ConvertedDate ORDER BY AVG_COMPOSITION DESC) AS AVG_COMPOSITION_RANK
    FROM CTE
),CTE2 AS
(
    SELECT 
        ConvertedDate,
        ROUND(AVG(AVG_COMPOSITION),2) AS [AVG OF AVG COMPOSITION FOR THE TOP 10 INTERESTS EACH MONTH]
    FROM CTE1
    WHERE AVG_COMPOSITION_RANK <= 10
    GROUP BY ConvertedDate
)

SELECT 
    ConvertedDate,
    [AVG OF AVG COMPOSITION FOR THE TOP 10 INTERESTS EACH MONTH]
FROM CTE2

(14 rows affected)

Total execution time: 00:00:00.391

ConvertedDate,AVG OF AVG COMPOSITION FOR THE TOP 10 INTERESTS EACH MONTH
2018-07-01,15.06
2018-08-01,10.82
2018-09-01,12.2
2018-10-01,13.67
2018-11-01,12.26
2018-12-01,13.22
2019-01-01,12.11
2019-02-01,12.34
2019-03-01,10.89
2019-04-01,9.55


4. What is the 3 month rolling average of the max average composition value from September 2018 to August 2019 and include the previous top ranking interests in the same output shown below.

In [28]:
WITH CTE AS 
(
    SELECT 
        ConvertedDate,
        T2.interest_name,
        AVG(T1.composition) OVER (PARTITION BY T1.ConvertedDate , T2.INTEREST_NAME ORDER BY ConvertedDate) AS [MAX AVG COMPOSITION]
    FROM filtered_data T1 JOIN Fresh_Segments.interest_map T2
        ON T1.interest_id = T2.id
), CTE1 AS 
(
    SELECT 
        ConvertedDate,
        interest_name,
        [MAX AVG COMPOSITION],
        DENSE_RANK() OVER (PARTITION BY ConvertedDate ORDER BY [MAX AVG COMPOSITION] DESC) AS AVG_COMPOSITION_RANK
    FROM CTE
),CTE2 AS 
(
    SELECT
        ConvertedDate,
        interest_name,
        [MAX AVG COMPOSITION],
        AVG_COMPOSITION_RANK
    FROM CTE1
    WHERE AVG_COMPOSITION_RANK = 1
),CTE3 AS 
(
    SELECT 
        ConvertedDate,
        interest_name,
        [MAX AVG COMPOSITION],
        AVG([MAX AVG COMPOSITION]) OVER (ORDER BY ConvertedDate ROWS BETWEEN CURRENT ROW AND 2 FOLLOWING) AS [3 MONTHS MOVING AVERAGE]
    FROM CTE2
),CTE4 AS
(
    SELECT
        ConvertedDate,
        interest_name,
        [MAX AVG COMPOSITION],
        [3 MONTHS MOVING AVERAGE],
        LAG(CONCAT(interest_name,': ',[MAX AVG COMPOSITION]),1,'NO INFO') OVER (ORDER BY ConvertedDate) AS [1 MONTH AGO],
        LAG(CONCAT(interest_name,': ',[MAX AVG COMPOSITION]),2,'NO INFO') OVER (ORDER BY ConvertedDate) AS [2 MONTHS AGO]
    FROM CTE3 
)

SELECT 
    ConvertedDate,
    interest_name,
    [MAX AVG COMPOSITION],
    ROUND([3 MONTHS MOVING AVERAGE],2) AS [3 MONTHS MOVING AVERAGE],
    [1 MONTH AGO],
    [2 MONTHS AGO]
FROM CTE4
WHERE ConvertedDate BETWEEN '2018-09-01' AND '2019-08-01'

(12 rows affected)

Total execution time: 00:00:00.386

ConvertedDate,interest_name,MAX AVG COMPOSITION,3 MONTHS MOVING AVERAGE,1 MONTH AGO,2 MONTHS AGO
2018-09-01,Work Comes First Travelers,18.18,19.3,Gym Equipment Owners: 13.9,Gym Equipment Owners: 18.82
2018-10-01,Work Comes First Travelers,20.28,20.31,Work Comes First Travelers: 18.18,Gym Equipment Owners: 13.9
2018-11-01,Work Comes First Travelers,19.45,19.88,Work Comes First Travelers: 20.28,Work Comes First Travelers: 18.18
2018-12-01,Work Comes First Travelers,21.2,19.53,Work Comes First Travelers: 19.45,Work Comes First Travelers: 20.28
2019-01-01,Work Comes First Travelers,18.99,16.67,Work Comes First Travelers: 21.2,Work Comes First Travelers: 19.45
2019-02-01,Work Comes First Travelers,18.39,14.01,Work Comes First Travelers: 18.99,Work Comes First Travelers: 21.2
2019-03-01,Luxury Boutique Hotel Researchers,12.64,10.39,Work Comes First Travelers: 18.39,Work Comes First Travelers: 18.99
2019-04-01,Luxury Bedding Shoppers,11.01,8.49,Luxury Boutique Hotel Researchers: 12.64,Work Comes First Travelers: 18.39
2019-05-01,Luxury Bedding Shoppers,7.53,7.22,Luxury Bedding Shoppers: 11.01,Luxury Boutique Hotel Researchers: 12.64
2019-06-01,Gym Equipment Owners,6.94,7.08,Luxury Bedding Shoppers: 7.53,Luxury Bedding Shoppers: 11.01
